In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
df =pd.read_csv('data_akbilgic.csv')
df.head()

,date,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,5-Jan-09,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524
1,6-Jan-09,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773
2,7-Jan-09,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
3,8-Jan-09,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
4,9-Jan-09,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802


In [3]:
df =df.drop('date', axis=1)

In [4]:
train_size = int(df.shape[0]*0.8)
df_train = df[:train_size]
df_test = df[train_size:]

df_train = df_train.values
df_test =df_test.values

In [5]:
# Receives the number of samples (batch_size) of size (n_steps) to extract
# from the time series, and outputs such a sample
def next_stock_batch(batch_size, n_steps, feat, n_features):
    t_min = 0
    t_max = feat.shape[0]
  
    # The inputs will be formed by 8 sequences taken from
    # 7 time series [ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU]
    x = np.zeros((batch_size,n_steps,n_features))
    
    # We want to predict the returns of the Istambul stock
    # taken into consideration the previous n_steps days
    y = np.zeros((batch_size,n_steps))

    # We chose batch_size random points from time series x-axis

    starting_points = np.random.randint(0,t_max-n_steps-1,size=batch_size)    
    #print(starting_points)
    #print(feat.shape)
    
    # We create the batches for x using all time series (8) between t and t+n_steps]
    for i, sp in enumerate(starting_points):
        x[i] = feat[sp: sp+n_steps]
        y[i] = feat[sp+1:sp+n_steps+1, 1]    
    # We create the batches for y using only one time series between t+1 and t+n_steps+1
    
    #Save on x and y the time series data sequence and the prediction sequence

    return x,y

# x,y =  next_stock_batch(batch_size=32,n_steps=10, feat=feat, n_features=9)
# print(y.shape)

In [7]:
class LSTM(nn.Module):
    def ___init__(self, input_size, hidden_size, num_layers, batch_size, seq_len):
        super(). __init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.batch_size =batch_size
        self.seq_len = seq_len
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first =True)

        self.fc1 = nn.Linear(self.batch_size*self.hidden_size, 1024)
        self.fc2 = nn.Linear(1024, self.batch_size*self.seq_len)

    def forward(self, x):
        h_0=torch.zeros((self.num_layers, self.batch_size, self.hidden_size ))

        lstm_out, h_n = self.lstm(x,h_0)
        last_hidden = h_n[-1]

        x = F.relu(last_hidden.flatten()) # added this line, you can activate also the last hidden layer, for better performance
        x = F.relu(self.fc1(x))
        out = self.fc2(x)
        return out

model = LSTM(9, 5, 10, 32, 10)

TypeError: Module.__init__() takes 1 positional argument but 6 were given